# Introduction
This project attempts to predict the classification of a _sentence_ into either a question or not a question

In [2]:
# Imports
import pandas as pd

In [0]:

import nltk.data;
from gensim.models import word2vec;
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;
import pandas as pd;
import numpy as np;
import os;
import re;
import logging;
import sqlite3;
import time;
import sys;
import multiprocessing;
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
import numpy

# Outline
Here, we will be going through five steps of data analysis & building models:
1. Collect Data
2. Clean Data
3. Explore & analyze Data
4. Build models
5. Evaluate

# 1.Collect Data
The data used in this project was collected from two different resources, one resource for data that were classify as sentences, and the other for data classify as questions.

The first is [OPUS - Wikipedia](http://opus.nlpl.eu/Wikipedia.php), many datasets are avialable at OPUS, we chose the wikipedia dataset being the most general set.

The other source for data that classify as questions is [this](https://github.com/vishaljain3991/cnn_topic_classification), we took all the data regardless of the domains.

In [3]:
# Here we need to add the steps we went through to merge two datasets together
# All the steps, like importing wiki, cleaning & filtering, then merging it with
# data from github.

# Untill finally we reach our dataset:
df = pd.read_csv('CPCS433Dataset.csv')
df.rename(columns = {'sentence ':'sentence'}, inplace = True)

# 2.Clean Data
The steps taken in order to clean the data:
* Change sentences to lower case

In [4]:
# Change sentences to lower case
df['sentence'] = df['sentence'].str.lower()

In [5]:
df.head()

,sentence,type
0,is 3hrill com brandable for a t shirt business...,Q
1,why effective app support is necessary for you...,Q
2,what business can be set up next to a car show...,Q
3,how should i report a vending machine competitor,Q
4,how can i retain younger employees with least ...,Q


In [6]:
df.tail()

,sentence,type
16693,some republicans on the other hand have respon...,S
16694,jeanjacques gosso gosso born 15 march 1983 in ...,S
16695,aristide benoît zogbo born december 30 1981 in...,S
16696,siaka tiéné born 22 february 1982 is an ivoria...,S
16697,he is also an ivory coast international,S


In [7]:
# Take 2000 from head
small_df = df.iloc[:2000, :]
# Take 2000 from tail
small_df2 = df.iloc[-2000:,:]
small_df2.head()

,sentence,type
14698,her victory in the 400 m event left silver med...,S
14699,she improved her record to 43610 at guayaquil ...,S
14700,she also held the world record in the 200 m in...,S
14701,she also collected a silver medal in the 400 m...,S
14702,she repeated her haul at the 1982 world champi...,S


In [8]:
# combine them
dataframe = pd.concat([small_df,small_df2])
len(dataframe)

4000

In [9]:
# Shuffle data
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,sentence,type
0,a few days after its release italian director ...,S
1,why does nuvaring needs to be refrigerated,Q
2,are there any arabic books about designing gam...,Q
3,surprisingly he is good at making cookies and ...,S
4,he used to answer the question of whether he w...,S
5,is the book by george jordac the voice of huma...,Q
6,its name is dedicated to bernardo de monteagud...,S
7,why are some christians opposed to the technol...,Q
8,montreal swept the rangers to advance to the f...,S
9,william ii also sent expeditions to the region...,S


In [21]:
import nltk
# import nltk.data;
# nltk.download('punkt')
from nltk.tokenize import word_tokenize

# The frequency distribution of the words
sentences = dataframe.sentence.str.cat(sep=' ')

tokens = nltk.word_tokenize(sentences)

freqDist = nltk.FreqDist(tokens)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/anfal/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['sentence']), axis=1)
df.head()

In [22]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

ModuleNotFoundError: No module named 'gensim'

In [0]:
# download the model and return as object ready for use
model_glove_twitter = api.load("glove-twitter-25")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model_gigaword = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model_gigaword.wv.most_similar("airplane",topn=10)
# Other pretrained:
# word2vec-google-news-300
# word2vec-ruscorpora-300

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('plane', 0.8651753067970276),
 ('jet', 0.8155897259712219),
 ('aircraft', 0.7861484289169312),
 ('airplanes', 0.7662244439125061),
 ('airliner', 0.7646356821060181),
 ('jetliner', 0.7469576597213745),
 ('flight', 0.733950138092041),
 ('planes', 0.733876645565033),
 ('flying', 0.7300053834915161),
 ('crash', 0.7099080085754395)]

In [0]:
model_gigaword.wv['hello']

In [0]:
sentence = ['london', "is", "the", "capital", "of", "great", "britain"]
vectors = [model_gigaword[w] for w in sentence]

In [0]:
! python -m spacy download en_core_web_md

     |████████████████████████████████| 95.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126237 sha256=fc8a37d8725ce51dfe0a6b9c82fcbbd763ac74e0aa80df8da726f824a548810b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1y1g6ycg/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
# Access vectors for specific words with a keyed lookup:
vector = model_gigaword['easy']
# see the shape of the vector (300,)
vector.shape
# Processing sentences is not as simple as with Spacy:
vectors = [model_gigaword[x] for x in ["this", "is", "some", "text"]]
vec = []
for v in vectors:
  vec = np.concatenate((vec, v), axis=0)
  print(v)
  print("Next:")

print(vec.shape)

[-0.57058   0.44183   0.70102  -0.41713  -0.34058   0.02339  -0.071537
  0.48177  -0.013121  0.16834  -0.13389   0.040626  0.15827  -0.44342
 -0.019403 -0.009661 -0.046284  0.093228 -0.27331   0.2285    0.33089
 -0.36474   0.078741  0.3585    0.44757  -0.2299    0.18077  -0.6265
  0.053852 -0.29154  -0.4256    0.62903   0.14393  -0.046004 -0.21007
  0.48879  -0.057698  0.37431  -0.030075 -0.34494  -0.29702   0.15095
  0.28248  -0.16578   0.076131 -0.093016  0.79365  -0.60489  -0.18874
 -1.0173    0.31962  -0.16344   0.54177   1.1725   -0.47875  -3.3842
 -0.081301 -0.3528    1.8372    0.44516  -0.52666   0.99786  -0.32178
  0.033462  1.1783   -0.072905  0.39737   0.26166   0.33111  -0.35629
 -0.16558  -0.44382  -0.14183  -0.37976   0.28994  -0.029114 -0.35169
 -0.27694  -1.344     0.19555   0.16887   0.040237 -0.80212   0.23366
 -1.3837   -0.023132  0.085395 -0.74051  -0.073934 -0.58838  -0.085735
 -0.10525  -0.51571   0.15038  -0.16694  -0.16372  -0.22702  -0.66102
  0.47197   0.37253 

In [0]:
model_gigaword.vector_size

100

In [0]:
def sentence_into_vec(df):
  
  # Iterate over each row
  for index, row in df.iterrows():
    # Extract cell
    sent = row['tokenized_sents']

    # Vectorize
    try:
      vectors = [model_gigaword[x] for x in sent]
    except:
      print("Word not found in vocab.")

    vec = []
    for v in vectors:
      vec = np.concatenate((vec, v), axis=0)
    print(vec.shape)

    # Save vector over cell
    row['tokenized_sents'] = vec


In [0]:
df.head()

,sentence,type,tokenized_sents
0,with technology innovation and new business mo...,Q,"[-0.4360800087451935, 0.39103999733924866, 0.5..."
1,the landscapes of earth during the quaternary ...,S,"[-0.03819400072097778, -0.24487000703811646, 0..."
2,lee has also appeared as harry tang chucks off...,S,"[-0.03819400072097778, -0.24487000703811646, 0..."
3,which are the best books for amie section a exam,Q,"[0.030239999294281006, 0.4460600018501282, 0.4..."
4,what is the best way to prepare a chicken mars...,Q,"[-0.1518000066280365, 0.38409000635147095, 0.8..."


In [0]:
df['tokenized_sents'].iloc[0]

,sentence,type,tokenized_sents
0,why do i get fat even though i don t eat much,Q,"[0.18449999392032623, 0.5146099925041199, 0.65..."
1,the montreal canadiens won their first stanley...,S,"[-0.03819400072097778, -0.24487000703811646, 0..."
2,bill durnan was the last goaltender in nhl his...,S,"[-0.10535000264644623, -0.02504800073802471, 0..."
3,paolo bianco born 20 august 1977 is an italian...,S,"[0.9362000226974487, -0.44703999161720276, -0...."
4,who has the best dropshipping service for hair...,Q,"[0.9362000226974487, -0.44703999161720276, -0...."
...,...,...,...
16693,what are the different possible strategies of ...,Q,"[-0.1518000066280365, 0.38409000635147095, 0.8..."
16694,how do i book a flat in rome for free for visa...,Q,"[-0.2376900017261505, 0.5939199924468994, 0.58..."
16695,i have recently published a book and want to g...,Q,"[-0.046539001166820526, 0.6196600198745728, 0...."
16696,coös county with two syllables frequently spe...,S,"[-0.046539001166820526, 0.6196600198745728, 0...."


In [0]:
max_shape = 0
for index, row in dataframe.iterrows():
    sent = row['tokenized_sents']
    current = sent.shape[0]
    if current > max_shape:
      max_shape = current
print(max_shape)


KeyError: ignored

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences


# First Encode label 
encoder = preprocessing.LabelEncoder()
df['type'] = encoder.fit_transform(df['type'])

features = df['tokenized_sents']
label = df['type']

featuresP = pad_sequences(features, padding='post', dtype='float32')


x_train, x_test, y_train, y_test = train_test_split(featuresP, label, train_size=0.8)

len(featuresP)

Using TensorFlow backend.


16698

In [0]:
#Train SVM Model
from sklearn import model_selection, svm
import timeit

start = timeit.default_timer()

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train,y_train)
predictions_SVM = SVM.predict(x_test)

stop = timeit.default_timer()



In [0]:
# Get accuracy
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

acc = accuracy_score(predictions_SVM, y_test)
print("Accuracy: ", acc)
print("Error Rate: ", 1 - acc)
print("Precision: ", precision_score(y_test, predictions_SVM))
print("Recall: ", recall_score(y_test, predictions_SVM))
print("Time: ",stop - start)
print("F1 = ", f1_score(y_test, predictions_SVM , average="macro"))
print("Confusion matrix", confusion_matrix(y_test, predictions_SVM))

Accuracy:  0.8263473053892215
Error Rate:  0.17365269461077848
Precision:  0.8730048577376822
Recall:  0.7601208459214501
Time:  1447.0216181690012
F1 =  0.8254155707825765
Confusion matrix [[1502  183]
 [ 397 1258]]


In [0]:
accuracy_score(y_train, SVM.predict(x_train))

0.8952687528073064